# Pipeline. 🤔📈

❗ Mediante el empleo de pipeline podemos determinar de manera sencilla el mejor modelo predictivo para nuestros datos. De este modo, le pasamos un parámetro con varios modelos y nos devolverá el que mejor resultados obtenga.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv("../../data/precio/dfclust.csv", index_col = 0)
df.head()

❗⚠ **SE REALIZARÁ UN PIPELINE PARA CADA CLUSTER**.

🤔 Para ello, en primer lugar se eliminarán todos los jugadores englobados en los clusters 1 y 2. Posteriormente, se realizará el mismo proceso que para el cluster 0 con los 1 y 2, eliminando en cada caso el resto de clusters.

🧐 En definitiva, después de realizar el pipeline para cada cluster, obtenemos que en todos ellos el mejor modelo es la **regresión lineal**.

1️⃣ Se eliminan los cluster excepto 1, en este caso se trabajará con el cluster 1.

In [3]:
df.drop(df[df['Cluster'] >= 1].index, inplace=True)

2️⃣ Se analizan datos estadísticos generales de cada uno de los componentes del dataframe.

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
edad,141.0,1.839716e+01,1.235657,16.0,18.0,18.0,19.0,21.0
Valoración_actual,141.0,6.573050e+01,1.974406,62.0,64.0,65.0,67.0,69.0
Progresión,141.0,8.162411e+01,1.754745,80.0,80.0,81.0,83.0,87.0
precio,141.0,2.095035e+06,650969.194041,1200000.0,1500000.0,2000000.0,2600000.0,3300000.0
Ataque,141.0,5.450355e+01,9.633738,27.0,50.0,57.0,61.0,71.0
Regate,141.0,6.887943e+01,4.301952,57.0,66.0,69.0,72.0,78.0
Aceleración,141.0,7.647518e+01,6.741748,59.0,71.0,77.0,82.0,92.0
Potencia_tiro,141.0,6.419149e+01,5.279225,48.0,61.0,65.0,68.0,78.0
Agresividad,141.0,4.688652e+01,10.295694,28.0,40.0,47.0,54.0,72.0
Defensa,141.0,2.926241e+01,9.804260,11.0,21.0,28.0,38.0,50.0


3️⃣ Se establecen las condiciones para la realización del pipeline. En este caso, se fijan 4 modelos distintos que se analizarán para obtener el que mejor métricas devuelva.

In [5]:
y = df["precio"]
X = df.drop(columns=["precio"])

In [6]:
pipe = Pipeline(steps=[('estimator', LinearRegression())])

In [7]:
params_grid = [{
                'estimator':[LinearRegression()],
                },
                {
                'estimator': [DecisionTreeRegressor()],
                'estimator__max_depth': [4, 5, 6, 7],
                'estimator__max_features': ["sqrt", "log2"],
                'estimator__min_samples_split': [10, 50, 100], 
                'estimator__min_samples_leaf':[1,2,3],
                },
                {'estimator':[RandomForestRegressor()],
                 'estimator__n_estimators':[50,100,200,300],
                 'estimator__max_depth':[4, 5, 6, 7, 9], 
                 'estimator__min_samples_split':[2,3,4], 
                 'estimator__min_samples_leaf':[1,2,3],
                },
                {'estimator':[KNeighborsRegressor()],
                 'estimator__n_neighbors':[3,5,6,7, 8, 9, 10],
                }

              ]

In [8]:
grid = GridSearchCV(pipe, params_grid, n_jobs= -1, verbose = 3)

In [9]:
%%time
res = grid.fit(X, y)

Fitting 5 folds for each of 260 candidates, totalling 1300 fits
Wall time: 1min 57s


4️⃣ Se obtiene que el modelo que mejor se ajusta a nuestros datos es la **regresión lineal**.

In [10]:
best_model = grid.best_params_
best_model

{'estimator': LinearRegression()}

5️⃣ Se muestran los resultados obtenidos.

In [11]:
results = pd.DataFrame(grid.cv_results_).head()
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator,param_estimator__max_depth,param_estimator__max_features,param_estimator__min_samples_leaf,param_estimator__min_samples_split,param_estimator__n_estimators,param_estimator__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003598,0.003005,0.005523,0.005395,LinearRegression(),NaN,NaN,NaN,NaN,NaN,NaN,{'estimator': LinearRegression()},-0.276198,0.827328,0.777239,0.779873,0.171848,0.456018,0.438823,1
1,0.004598,0.000489,0.003398,0.000490,DecisionTreeRegressor(),4,sqrt,1,10,NaN,NaN,"{'estimator': DecisionTreeRegressor(), 'estima...",-31.640392,-4.685279,-3.528986,-0.459067,-12.197905,-10.502326,11.251892,216
2,0.006597,0.002869,0.004997,0.000894,DecisionTreeRegressor(),4,sqrt,1,50,NaN,NaN,"{'estimator': DecisionTreeRegressor(), 'estima...",-20.954551,-0.211676,-0.012916,0.073653,-21.351088,-8.491315,10.339249,202
3,0.007995,0.003345,0.006396,0.003135,DecisionTreeRegressor(),4,sqrt,1,100,NaN,NaN,"{'estimator': DecisionTreeRegressor(), 'estima...",-20.954551,-3.930198,-1.505176,-21.103145,-22.729268,-14.044468,9.300884,229
4,0.006396,0.001854,0.004198,0.001599,DecisionTreeRegressor(),4,sqrt,2,10,NaN,NaN,"{'estimator': DecisionTreeRegressor(), 'estima...",-17.622022,-7.615833,-0.849655,-9.713761,-7.435796,-8.647414,5.385497,204


In [12]:
results = cross_validate(LinearRegression(), 
               X=X, y=y, scoring='neg_root_mean_squared_error', cv=10, verbose = 3,  n_jobs= -1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


In [13]:
rmse_ = abs(results["test_score"].mean())
rmse_

98486.9054602189